<a href="https://colab.research.google.com/github/rishubhkhurana/nlp/blob/main/attnmodels/AttentionFromBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libs

In [351]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import Multi30k
import spacy, random
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import time, math

## Book Keeping

In [2]:
!python -m spacy download en > /dev/null 2>&1
!python -m spacy download de > /dev/null 2>&1

## Dataset and Loaders for Type 1 Model

In [277]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [278]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [279]:
SRC = Field(tokenize=tokenize_en, lower=True, eos_token='<eos>', init_token='<sos>')
TRG = Field(tokenize=tokenize_de, lower=True, eos_token='<eos>', init_token='<sos>')

In [280]:
train_data, val_data, test_data = Multi30k.splits(exts=('.en','.de'), fields=(SRC, TRG))

In [281]:
#vars(train_data.examples[0])

In [325]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [326]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [327]:
train_dl, valid_dl, test_dl = BucketIterator.splits((train_data, val_data, test_data), batch_size = 64, device = device)

## Model Type 1

In [300]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dims, enc_hidden_dims, dec_hidden_dims, dropout=0.1):
        super().__init__()
        # saving variables
        self.hidden_dims = enc_hidden_dims
        self.embedder = nn.Embedding(vocab_size, embed_dims)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.GRU(embed_dims, enc_hidden_dims, bidirectional=True)
        self.bridge = nn.Linear(2*enc_hidden_dims, dec_hidden_dims)
    
    def forward(self, src):
        # src : [srclen, batch size]
        embedded = self.embedder(src)
        # embedded: [srclen, batch size, embed_dims]
        embedded = self.dropout(embedded)
        # pass through the rnn 
        outputs, hidden = self.rnn(embedded)
        # outputs : [srclen, batch size, enc_hidden_dims* num_directions]
        # hidden : [num_layers*num_directions, batch_size, enc_hidden_dims]
        hidden = torch.cat([hidden[-2,:,:], hidden[-1,:,:]], dim = 1)
        hidden = torch.tanh(self.bridge(hidden))
        return outputs, hidden.unsqueeze(0)

In [301]:
enc = Encoder(vocab_size=len(SRC.vocab), embed_dims=32, enc_hidden_dims=64, dec_hidden_dims=64)

In [302]:
enc

Encoder(
  (embedder): Embedding(5893, 32)
  (dropout): Dropout(p=0.1, inplace=False)
  (rnn): GRU(32, 64, bidirectional=True)
  (bridge): Linear(in_features=128, out_features=64, bias=True)
)

In [303]:
batch = next(iter(trn_dl))

In [288]:
enc(batch.src)[0].shape, enc(batch.src)[1].shape

(torch.Size([25, 64, 128]), torch.Size([1, 64, 64]))

In [289]:
class Attention(nn.Module):
    def __init__(self, enc_hidden_dims, dec_hidden_dims):
        super().__init__()
        self.energymodel = nn.Linear(2*enc_hidden_dims + dec_hidden_dims, dec_hidden_dims)
        self.valuemodel = nn.Linear(dec_hidden_dims, 1, bias=False)
    def forward(self, hidden, enc_outputs):
        # hidden: [batch size, dec_hidden_dims]
        # enc_outputs: [srclen, batch size, 2*enc_hidden_dims]
        src_len = enc_outputs.shape[0]
        hidden = hidden.permute((1,0,2))
        hidden = hidden.repeat(1, src_len, 1)
        energy = torch.cat([enc_outputs.permute((1,0,2)), hidden], dim = 2)
        # energy: [batch size, src len, 2*enc_hidden_dims + dec_hidden_dims]
        energy = torch.tanh(self.energymodel(energy))
        # energy : [batch size, src len, dec_hidden_dims]
        value = self.valuemodel(energy).squeeze(2)
        # value: [batch size, srclen]
        return F.softmax(value, dim=1)

In [290]:
attn = Attention(enc_hidden_dims=100, dec_hidden_dims=100)

In [304]:
hidden = torch.randn((1,32,100))
encoder_outputs = torch.randn((22,32,200))
values = attn(hidden, encoder_outputs)

In [305]:
values.shape

torch.Size([32, 22])

In [306]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dims, dec_hidden_dims, enc_hidden_dims, attention, dropout = 0.1):
        super().__init__()
        # saving variable dims
        self.output_dims = vocab_size
        self.hidden_dims = dec_hidden_dims
        # embedding for decoder
        self.embedder = nn.Embedding(vocab_size, embed_dims)
        self.dropout = nn.Dropout(dropout)
        # rnn for decoder, takes embeddings, attention based enc_hidden_dims, 
        self.rnn = nn.GRU((embed_dims + 2*enc_hidden_dims), dec_hidden_dims)
        # store the attention model to be used 
        self.attn = attention
        # classifier
        self.classifier = nn.Linear(2*enc_hidden_dims + embed_dims + dec_hidden_dims, vocab_size)

    def forward(self, decoder_input, hidden, encoder_outputs):
        # decoder_input: [batch size]
        # hidden: [batch size, dec_hidden_dims]
        # encoder_outputs: [src len, batch size, 2*enc_hidden_dims]
        decoder_input = decoder_input.unsqueeze(0)
        # decoder_input: [1, batch size]
        embedded = self.dropout(self.embedder(decoder_input))
        # embedded: [1, batch size, embed_dims]
        # walk through the attention model to compute the attention weights 
        attn_weights = self.attn(hidden, encoder_outputs)
        # attn_weights: [batch size, src len]
        attn_weights = attn_weights.unsqueeze(1)
        # attn_weights: [batch size, 1, src len]
        weighted_encoder = torch.bmm(attn_weights, encoder_outputs.permute((1,0,2)))
        # weighted_encoder: [batch size, 1, 2*enc_hidden_dims]
        weighted_encoder = weighted_encoder.permute((1, 0, 2))
        # weighted_encoder: [1, batch size, 2*enc_hidden_dims]
        # walk through the rnn 
        outputs, hidden = self.rnn(torch.cat([embedded, weighted_encoder], dim=2), hidden)
        # outputs: [1, batch size, dec_hidden_dims], hidden: [batch size, dec_hidden_dims]
        # walk through the classifier
        classifier_outputs = torch.cat([embedded.squeeze(0), weighted_encoder.squeeze(0), hidden.squeeze(0)], dim = 1)
        classifier_outputs = self.classifier(classifier_outputs)
        # classifier_outputs: [ batch size, vocab size]
        return classifier_outputs, hidden

In [307]:
dec = Decoder(len(TRG.vocab), 100, 100, 100, attn)

In [308]:
dec

Decoder(
  (embedder): Embedding(7855, 100)
  (dropout): Dropout(p=0.1, inplace=False)
  (rnn): GRU(300, 100)
  (attn): Attention(
    (energymodel): Linear(in_features=300, out_features=100, bias=True)
    (valuemodel): Linear(in_features=100, out_features=1, bias=False)
  )
  (classifier): Linear(in_features=400, out_features=7855, bias=True)
)

In [335]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device='cuda'):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_fratio = 0.5):
        # src: [src len, batch size]
        # trg: [trg len, batch size]
        trglen = trg.shape[0]
        bs = trg.shape[1]
        vocab_size = self.decoder.output_dims
        outputs = torch.zeros((trglen, bs, vocab_size)).to(self.device)
        enc_outputs, hidden = self.encoder(src)
        # enc_outputs: [srclen, batch size, enc_hidden_dims*2]
        # hidden: [1, batch size, dec_hidden_dims]
        decoder_inp = trg[0]
        for t in range(1, trglen):
            decoder_inp, hidden = self.decoder(decoder_inp, hidden, enc_outputs)
            # decoder_inp: [batch size, vocab_size], hidden: [batch size, dec_hidden_dims]
            # add decoder output to all the outputs tensor
            outputs[t,:,:] = decoder_inp
            # teacher force the input based on force ratio
            if random.random() < teacher_fratio:
                decoder_inp = trg[t]
            else:
                decoder_inp = decoder_inp.argmax(dim=1)
        return outputs

In [310]:
enc_dims = 512
dec_dims = 512
enc_embed_dims = 256
dec_embed_dims = 256
enc_dropout = 0.5
dec_dropout = 0.5
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)


In [59]:
enc_model = Encoder(vocab_size=src_vocab_size, enc_hidden_dims=enc_dims, 
                    embed_dims = enc_embed_dims, dec_hidden_dims = dec_dims, dropout = enc_dropout)
attn_model = Attention(enc_hidden_dims=enc_dims, dec_hidden_dims=dec_dims)
dec_model = Decoder(vocab_size = trg_vocab_size, embed_dims = dec_embed_dims, dec_hidden_dims = dec_dims, 
                    enc_hidden_dims = enc_dims, dropout = dec_dropout, attention = attn_model)
smodel = Seq2Seq(encoder = enc_model, decoder = dec_model)

In [54]:
# get batch and test all models
batch = next(iter(trn_dl))

In [55]:
enc_outputs, enc_hidden = enc_model(batch.src)
print(f"Encoder outputs size: {enc_outputs.shape}, Encoder hidden dims: {enc_hidden.shape}")

Encoder outputs size: torch.Size([26, 64, 1024]), Encoder hidden dims: torch.Size([1, 64, 512])


In [56]:
attn_outputs = attn_model(enc_hidden, enc_outputs)
print(f"Attention outputs size: {attn_outputs.shape}")

Attention outputs size: torch.Size([64, 26])


In [61]:
dec_output, dec_hidden = dec_model(batch.trg[0], enc_hidden, enc_outputs)
print(f"Decoder outputs size: {dec_output.shape}, Decoder hidden dims: {dec_hidden.shape}")

Decoder outputs size: torch.Size([64, 7855]), Decoder hidden dims: torch.Size([1, 64, 512])


In [66]:
seq_outputs = smodel(batch.src, batch.trg)
print(f"Seq 2 Seq size: {seq_outputs.shape}")

Seq 2 Seq size: torch.Size([27, 64, 7855])


## Training Utils Type 1

In [271]:
def init_weights(model):
    for name, param in model.named_parameters():
        if name=='weight':
            nn.init.normal_(param.data, mean=0, std=0.01)
        elif name=='bias':
            nn.init.constant_(param.data, 0)


In [272]:
def count_parameters(model):
    total_params=0
    for param in model.parameters():
        total_params+=param.numel()
    return total_params

In [341]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [342]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [343]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training Model Type 1

In [352]:
enc_dims = 512
dec_dims = 512
enc_embed_dims = 256
dec_embed_dims = 256
enc_dropout = 0.5
dec_dropout = 0.5
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)

In [353]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [354]:
enc_model = Encoder(vocab_size=src_vocab_size, enc_hidden_dims=enc_dims, 
                    embed_dims = enc_embed_dims, dec_hidden_dims = dec_dims, dropout = enc_dropout).to(device)
attn_model = Attention(enc_hidden_dims=enc_dims, dec_hidden_dims=dec_dims).to(device)
dec_model = Decoder(vocab_size = trg_vocab_size, embed_dims = dec_embed_dims, dec_hidden_dims = dec_dims, 
                    enc_hidden_dims = enc_dims, dropout = dec_dropout, attention = attn_model).to(device)
model = Seq2Seq(encoder = enc_model, decoder = dec_model, device = device).to(device)

In [355]:
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedder): Embedding(5893, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(256, 512, bidirectional=True)
    (bridge): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (embedder): Embedding(7855, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(1280, 512)
    (attn): Attention(
      (energymodel): Linear(in_features=1536, out_features=512, bias=True)
      (valuemodel): Linear(in_features=512, out_features=1, bias=False)
    )
    (classifier): Linear(in_features=1792, out_features=7855, bias=True)
  )
)

In [356]:
print(f"Total Model parameters: {count_parameters(model):,}")

Total Model parameters: 24,036,783


In [357]:
opt = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [359]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dl, opt, criterion, CLIP)
    valid_loss = evaluate(model, valid_dl, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_type1.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 36s
	Train Loss: 4.537 | Train PPL:  93.416
	 Val. Loss: 4.141 |  Val. PPL:  62.892
Epoch: 02 | Time: 3m 34s
	Train Loss: 3.279 | Train PPL:  26.552
	 Val. Loss: 3.312 |  Val. PPL:  27.450
Epoch: 03 | Time: 3m 34s
	Train Loss: 2.593 | Train PPL:  13.365
	 Val. Loss: 3.098 |  Val. PPL:  22.152
Epoch: 04 | Time: 3m 32s
	Train Loss: 2.141 | Train PPL:   8.505
	 Val. Loss: 3.006 |  Val. PPL:  20.210
Epoch: 05 | Time: 3m 31s
	Train Loss: 1.793 | Train PPL:   6.004
	 Val. Loss: 3.034 |  Val. PPL:  20.772
Epoch: 06 | Time: 3m 32s
	Train Loss: 1.566 | Train PPL:   4.786
	 Val. Loss: 3.076 |  Val. PPL:  21.680
Epoch: 07 | Time: 3m 31s
	Train Loss: 1.410 | Train PPL:   4.096
	 Val. Loss: 3.128 |  Val. PPL:  22.821
Epoch: 08 | Time: 3m 31s
	Train Loss: 1.263 | Train PPL:   3.537
	 Val. Loss: 3.237 |  Val. PPL:  25.449
Epoch: 09 | Time: 3m 31s
	Train Loss: 1.155 | Train PPL:   3.174
	 Val. Loss: 3.282 |  Val. PPL:  26.642
Epoch: 10 | Time: 3m 30s
	Train Loss: 1.060 | Train PPL

In [328]:
batch = next(iter(train_dl))